In [1]:
##########################################################################################################
# VERSION  	DESARROLLADOR             FECHA        DESCRIPCION
# -------------------------------------------------------------
#  1        Walter Albites Azarte     29/11/2022   Curso PySpark Entorno Local - Dataframe
##########################################################################################################

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.1.2-bin-hadoop2.7'

In [3]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf= pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark=SparkSession(sc)

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [5]:
#Crear Dataframe Spark Universidades
schemaUniversidad = StructType([
    StructField("LICENCIAMIENTO", StringType(),True),
    StructField("UNIVERSIDAD", StringType(),True),
    StructField("CARRERAS", StringType(),True)
])

dataUniversidad = ([
                ("Res. N° 036-2018-SUNEDU/CD","Universidad Nacional Mayor de San Marcos","Derecho##Medicina##Licenciatura"),
                ("Res. N° 073-2017-SUNEDU/CD","Universidad Nacional de Ingeniería","Ingenieria Sistemas##Ingenieria Civil##Ingenieria de Minas##Ingenieria Industrial")
               ])

df_universidad=spark.createDataFrame(dataUniversidad,schema=schemaUniversidad)
df_universidad.show(5,False)

+--------------------------+----------------------------------------+---------------------------------------------------------------------------------+
|LICENCIAMIENTO            |UNIVERSIDAD                             |CARRERAS                                                                         |
+--------------------------+----------------------------------------+---------------------------------------------------------------------------------+
|Res. N° 036-2018-SUNEDU/CD|Universidad Nacional Mayor de San Marcos|Derecho##Medicina##Licenciatura                                                  |
|Res. N° 073-2017-SUNEDU/CD|Universidad Nacional de Ingeniería      |Ingenieria Sistemas##Ingenieria Civil##Ingenieria de Minas##Ingenieria Industrial|
+--------------------------+----------------------------------------+---------------------------------------------------------------------------------+



In [6]:
#explode
df_universidad_carrera=df_universidad.select('*',
                      explode(split(col('CARRERAS'),'##')).alias('CARRERA')
                     ).drop('CARRERAS')

df_universidad_carrera.show(20,False)

+--------------------------+----------------------------------------+---------------------+
|LICENCIAMIENTO            |UNIVERSIDAD                             |CARRERA              |
+--------------------------+----------------------------------------+---------------------+
|Res. N° 036-2018-SUNEDU/CD|Universidad Nacional Mayor de San Marcos|Derecho              |
|Res. N° 036-2018-SUNEDU/CD|Universidad Nacional Mayor de San Marcos|Medicina             |
|Res. N° 036-2018-SUNEDU/CD|Universidad Nacional Mayor de San Marcos|Licenciatura         |
|Res. N° 073-2017-SUNEDU/CD|Universidad Nacional de Ingeniería      |Ingenieria Sistemas  |
|Res. N° 073-2017-SUNEDU/CD|Universidad Nacional de Ingeniería      |Ingenieria Civil     |
|Res. N° 073-2017-SUNEDU/CD|Universidad Nacional de Ingeniería      |Ingenieria de Minas  |
|Res. N° 073-2017-SUNEDU/CD|Universidad Nacional de Ingeniería      |Ingenieria Industrial|
+--------------------------+----------------------------------------+-----------

In [7]:
#Funcion exportacion a CSV
import os
import shutil
def write_df(spark, flag_csv, df, root_path, output_name):
    if flag_csv:
        df.coalesce(1).write.mode('overwrite').option('header', 'true').option('delimiter', ';').option('encoding', 'UTF-8').csv(root_path + output_name + '_csv')
        fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
        list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(root_path + output_name + '_csv'))
        file_name = [file.getPath().getName() for file in list_status if file.getPath().getName().startswith('part-')][0]
        file = root_path + output_name + '_csv' + '/' + file_name
        file_new = root_path + output_name + ".csv"
        try:
            os.rename(file, file_new)
        except FileExistsError:
            os.remove(file_new)
            os.rename(file, file_new)
        shutil.rmtree(root_path + output_name + '_csv')
    df.write.mode("overwrite").parquet(root_path + output_name)
    return df

In [8]:
#Exportacion a CSV
flag_csv = True
Ruta_Output_csv = 'C:/Users/walbi/Documents/'
#Exportacion a CSV con Funcion
df = write_df(
    spark,
    flag_csv,
    df_universidad_carrera,
    Ruta_Output_csv,
    'df_universidad_carrera'
)

In [9]:
#Leer un csv correctamente
df_universidadades_lectura = spark.read.options(header='True',inferSchema='True',delimiter=';',escape='"',quote='"',multiLine=True).\
csv(Ruta_Output_csv + 'df_universidad_carrera.csv')
df_universidadades_lectura.show()

+--------------------+--------------------+--------------------+
|      LICENCIAMIENTO|         UNIVERSIDAD|             CARRERA|
+--------------------+--------------------+--------------------+
|Res. N° 036-2018-...|Universidad Nacio...|             Derecho|
|Res. N° 036-2018-...|Universidad Nacio...|            Medicina|
|Res. N° 036-2018-...|Universidad Nacio...|        Licenciatura|
|Res. N° 073-2017-...|Universidad Nacio...| Ingenieria Sistemas|
|Res. N° 073-2017-...|Universidad Nacio...|    Ingenieria Civil|
|Res. N° 073-2017-...|Universidad Nacio...| Ingenieria de Minas|
|Res. N° 073-2017-...|Universidad Nacio...|Ingenieria Indust...|
+--------------------+--------------------+--------------------+



In [10]:
#Funcion Autoincrement
def add_identify (
    inputDF,
    column_identify_name="Id",
    identity_start=0
):
    rpt = inputDF.withColumn(
        'monotonically_increasing_id',
        monotonically_increasing_id()
    )
    rpt = rpt.withColumn(
        column_identify_name,
        row_number().over(
            Window.orderBy(col('monotonically_increasing_id'))
        )
    )
    if (identity_start != 0):
        rpt = rpt.withColumn(column_identify_name, col(column_identify_name) + identity_start)
    return rpt.select( [column_identify_name] + inputDF.columns )

In [11]:
df_universidadades_identify = add_identify(df_universidad_carrera,'ID',0)
df_universidadades_identify.show()

+---+--------------------+--------------------+--------------------+
| ID|      LICENCIAMIENTO|         UNIVERSIDAD|             CARRERA|
+---+--------------------+--------------------+--------------------+
|  1|Res. N° 036-2018-...|Universidad Nacio...|             Derecho|
|  2|Res. N° 036-2018-...|Universidad Nacio...|            Medicina|
|  3|Res. N° 036-2018-...|Universidad Nacio...|        Licenciatura|
|  4|Res. N° 073-2017-...|Universidad Nacio...| Ingenieria Sistemas|
|  5|Res. N° 073-2017-...|Universidad Nacio...|    Ingenieria Civil|
|  6|Res. N° 073-2017-...|Universidad Nacio...| Ingenieria de Minas|
|  7|Res. N° 073-2017-...|Universidad Nacio...|Ingenieria Indust...|
+---+--------------------+--------------------+--------------------+

